In [1]:
import os
import re

import numpy as np
import nibabel as nib

In [2]:
img_path = '/home/yutongx/src_data/images/'
bbox_path = '/home/yutongx/src_data/bbox/'
save_path = '/home/yutongx/FeatureNet/masks/'

resize = 64

In [6]:
idx = lambda name: re.sub(r'\D', '', name)
get_names = lambda path: sorted([os.path.join(path, name) for name in os.listdir(path)], key=idx)

def crop(image, bbox, length):
    start = lambda center, length: int(np.floor(center - length/2))
    end = lambda center, length: int(np.floor(center + length/2))

    start_crop = lambda center, length: int(max(start(center, length), 0))
    end_crop = lambda center, length, max_len: int(min(end(center, length), max_len))

    zc, yc, xc, dz, dy, dx = bbox

    st = {'zs': start_crop(zc, length),
        'zt': end_crop(zc, length, image.shape[2]),
        'ys': start_crop(yc, length),
        'yt': end_crop(yc, length, image.shape[1]),
        'xs': start_crop(xc, length),
        'xt': end_crop(xc, length, image.shape[0])}
    
    return st

def foo(img_names, bbox_names, label_name, resize, save_path):
    bboxes_all = []
    for bbox_name in bbox_names:
        bboxes_src = np.load(bbox_name, allow_pickle=True)
        bboxes_all.append(bboxes_src[label_name])
    
    for img_name, bboxes in zip(img_names, bboxes_all):
        img = nib.load(img_name).get_fdata()#H*W*D
        mask = np.zeros(img.shape).astype(np.uint8)
        for index in range(bboxes.shape[0]):
            bbox = bboxes[index]
            st = crop(img, bbox, resize)
            mask[st['xs']:st['xt'], st['ys']:st['yt'], st['zs']:st['zt']] = 2048
 
        mask_name = ''.join((idx(img_name), '_', label_name, '.nii.gz'))
        nib.save(nib.Nifti1Image(mask, affine=None), os.path.join(save_path, mask_name))

In [4]:
img_train_names = get_names(os.path.join(img_path, 'train'))
bbox_train_names = get_names(os.path.join(bbox_path, 'train'))

img_val_names = get_names(os.path.join(img_path, 'val'))
bbox_val_names = get_names(os.path.join(bbox_path, 'val'))

save_train_path = os.path.join(save_path, 'train')
save_val_path = os.path.join(save_path, 'val')

In [ ]:
foo(img_train_names, bbox_train_names, 'gt_pos', resize, save_train_path)
foo(img_train_names, bbox_train_names, 'rpn_pos', resize, save_train_path)
foo(img_train_names, bbox_train_names, 'rpn_neg', resize, save_train_path)

In [ ]:
foo(img_val_names, bbox_val_names, 'gt_pos', resize, save_val_path)
foo(img_val_names, bbox_val_names, 'rpn_pos', resize, save_val_path)
foo(img_val_names, bbox_val_names, 'rpn_neg', resize, save_val_path)